# Homework 1 

In this homework I used some information from the internet, mostly technical details of using functions and objects of pandas or sklearn. The sources include official documentation, forums and chatGPT

In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.linalg import norm
import numpy as np

In [72]:
df = pd.read_csv("./PetersonBarney/verified_pb.data", delimiter='\t', header=None)
df

,0,1,2,3,4,5,6,7
0,1,1,1,IY,160.0,240.0,2280.0,2850.0
1,1,1,1,IY,186.0,280.0,2400.0,2790.0
2,1,1,2,IH,203.0,390.0,2030.0,2640.0
3,1,1,2,IH,192.0,310.0,1980.0,2550.0
4,1,1,3,EH,161.0,490.0,1870.0,2420.0
...,...,...,...,...,...,...,...,...
1515,3,76,8,UH,322.0,610.0,1550.0,3400.0
1516,3,76,9,UW,345.0,520.0,1250.0,3460.0
1517,3,76,9,UW,334.0,500.0,1140.0,3380.0
1518,3,76,10,ER,308.0,740.0,1850.0,2160.0


In [73]:
df.columns = ['gender', 'speaker', 'phoneme', 'phoneme_ascii', 'F0', 'F1', 'F2', 'F3']
df

,gender,speaker,phoneme,phoneme_ascii,F0,F1,F2,F3
0,1,1,1,IY,160.0,240.0,2280.0,2850.0
1,1,1,1,IY,186.0,280.0,2400.0,2790.0
2,1,1,2,IH,203.0,390.0,2030.0,2640.0
3,1,1,2,IH,192.0,310.0,1980.0,2550.0
4,1,1,3,EH,161.0,490.0,1870.0,2420.0
...,...,...,...,...,...,...,...,...
1515,3,76,8,UH,322.0,610.0,1550.0,3400.0
1516,3,76,9,UW,345.0,520.0,1250.0,3460.0
1517,3,76,9,UW,334.0,500.0,1140.0,3380.0
1518,3,76,10,ER,308.0,740.0,1850.0,2160.0


First, let's create the baseline model. We will first divide the dataset into training, validation and test with the 80/10/10 ratio. We then create our simple baseline model which will work like this: we will only use (F0, F1, F2, F3) and treat it as a vector in a 4-dimensional space. We first find the center of gravity for each phoneme and for every test sample we will find the closest out of these centers. 

In [74]:
X = df[['F0', 'F1', 'F2', 'F3']]
y = df['phoneme']
X,y

(         F0     F1      F2      F3
 0     160.0  240.0  2280.0  2850.0
 1     186.0  280.0  2400.0  2790.0
 2     203.0  390.0  2030.0  2640.0
 3     192.0  310.0  1980.0  2550.0
 4     161.0  490.0  1870.0  2420.0
 ...     ...    ...     ...     ...
 1515  322.0  610.0  1550.0  3400.0
 1516  345.0  520.0  1250.0  3460.0
 1517  334.0  500.0  1140.0  3380.0
 1518  308.0  740.0  1850.0  2160.0
 1519  328.0  660.0  1830.0  2200.0
 
 [1520 rows x 4 columns],
 0        1
 1        1
 2        2
 3        2
 4        3
         ..
 1515     8
 1516     9
 1517     9
 1518    10
 1519    10
 Name: phoneme, Length: 1520, dtype: int64)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state=25)

In [76]:
df_train = df.loc[X_train.index]

In [77]:
CoGs = []
for i in range(df.phoneme.nunique()):
    CoGs.append([ df_train.query(f"phoneme == {i+1}")[f'F{j}'].mean() for j in range(4) ])

In [78]:
def predict_phoneme(Fs, CoGs):
    CoGs = [np.array(x) for x in CoGs]
    minidx = 0
    minval = norm(Fs - CoGs[0])
    for i in range(1,len(CoGs)):
        if norm(Fs - CoGs[i]) < minval:
            minidx = i
            minval = norm(Fs - CoGs[1])
    return minidx+1

In [79]:
table_test = X_test.to_numpy()
label_test = y_test.to_numpy()

table_test.shape,lable_test.shape

((152, 4), (152,))

In [80]:
acc = 0
for i in range(table_test.shape[0]):
    pred = predict_phoneme(table_test[i], CoGs)
    if pred == label_test[i]:
        acc += 1 / table_test.shape[0]
print(acc)        

0.23684210526315772


Now we are going to train a more sophisticated model. I chose randomforest from scikit learn for this exercise 

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [82]:
model = RandomForestClassifier(n_estimators=100, random_state=25)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=25)

In [83]:
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.8947368421052632
